In [ ]:
import cPickle as pickle
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer

import xgboost as xgb

from sklearn.preprocessing import label_binarize
from sklearn.cross_validation import train_test_split

from sklearn.metrics import roc_auc_score, roc_curve, classification_report, auc

from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

stemmer = SnowballStemmer("english")

In [ ]:
# See nice example from 
# https://github.com/dmlc/xgboost/blob/master/demo/multiclass_classification/train.py

# Also RunMutliclassOneVRest

In [ ]:
df = pickle.load(open('../Data/labeledhate_5cats.p', 'rb'))

In [ ]:
X = df.body
y = df.label

In [ ]:
# Binarize the output
y = label_binarize(y, classes=['NotHate', 'SizeHate', 'GenderHate', 'RaceHate', 'ReligionHate'])
n_classes = y.shape[1]

In [ ]:
classes=['NotHate', 'SizeHate', 'GenderHate', 'RaceHate', 'ReligionHate']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#### Use Count Vectorizer

In [ ]:
countv = CountVectorizer(decode_error = 'ignore', stop_words = 'english', tokenizer=None)

In [ ]:
#Training vectors
countv_fit_X_train = countv.fit_transform(X_train)
#test vectora
countv_fit_X_test = countv.transform(X_test)

In [ ]:
#  .fit(countv_fit_X_train, y_train)

In [ ]:
xg_train = xgb.DMatrix(countv_fit_X_train, label=y_train)
xg_test = xgb.DMatrix(countv_fit_X_test, label=y_test)

In [ ]:
# Set up xboost parameters
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 6


watchlist = [ (xg_train, 'train'), (xg_test, 'test') ]
num_round = 5
bst = xgb.train(param, xg_train, num_round, watchlist );
# get prediction
pred = bst.predict( xg_test );

print ('predicting, classification error=%f' % (sum( int(pred[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) ))

# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bst = xgb.train(param, xg_train, num_round, watchlist );
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
yprob = bst.predict( xg_test ).reshape( test_Y.shape[0], 6 )
ylabel = np.argmax(yprob, axis=1)

print ('predicting, classification error=%f' % (sum( int(ylabel[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) ))
